In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
from langchain_openai import ChatOpenAI
import requests
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import re

llm = ChatOpenAI(
    model='gpt-4o-mini',
    base_url= os.getenv("url"),  
    api_key= os.getenv("api_key"),  
    max_tokens=1000 
)

# User location and search radius
user_location = (13.7563, 100.5018)  # Example: Bangkok
radius = 5  # Radius in kilometers

In [ ]:
# สร้างโมเดล Pydantic สำหรับการจัดการข้อมูลคำสำคัญที่ได้จาก LLM
class SearchKeyword(BaseModel):
    keyword: str = Field(description="คำสำคัญที่ได้จากการประมวลผลคำถาม") # เพิ่มเรืองการ promt

# Clean keyword function
def clean_keyword(keyword: str) -> str:
    unwanted_words = ["แถวนี้", "ใกล้ฉัน", "ใกล้", "ช่วยหา"]
    for word in unwanted_words:
        keyword = keyword.replace(word, "")
    return re.sub(r'\s+', ' ', keyword).strip()

# Search function for Logdo Map API
def search_logdo_map_api(keyword, user_location, radius):
    base_url = "https://search.longdo.com/mapsearch/json/search?"
    params = {
        'keyword': keyword,
        'lon': user_location[1],
        'lat': user_location[0],
        'span': radius,
        'locale': 'th',
        'key': os.getenv("key_longdo")
    }
    response = requests.get(base_url, params=params)
    return response.json() if response.status_code == 200 else None

# Process user query
def process_user_query(user_query, user_location, radius):
    # Parse keyword from LLM
    parser = JsonOutputParser(pydantic_object=SearchKeyword)
    prompt = PromptTemplate(
        template="""\
## คุณมีหน้าที่กรองคำสำคัญจากคำขอผู้ใช้. # เพิ่มเรืองการ promt

# คำขอผู้ใช้: {user_query}

# Your response should be structured as follows:
{format_instructions}
""",
        input_variables=["user_query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | llm | parser
    event = chain.invoke({"user_query": user_query, "user_location": user_location})

    if event:
        keyword = event.get('keyword', '')
        cleaned_keyword = clean_keyword(keyword)
        if cleaned_keyword:
            places_from_api = search_logdo_map_api(cleaned_keyword, user_location, radius)
            return cleaned_keyword, places_from_api
    return None, None


In [12]:
user_query = "บอกรายชื่อเซเว่นแถวนี้ให้หน่อย"
print(f"คำขอผู้ใช้: {user_query}")
print(f"ตำแหน่งผู้ใช้: {user_location}")
print(f"รัศมี: {radius} กิโลเมตร")

คำขอผู้ใช้: บอกรายชื่อเซเว่นแถวนี้ให้หน่อย
ตำแหน่งผู้ใช้: (13.7563, 100.5018)
รัศมี: 5 กิโลเมตร


In [13]:
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("รายชื่อสถานที่:")
        for place in places_from_api["data"]:
            print(place.get("name", "ไม่ทราบชื่อ"))
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("ไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น
รายชื่อสถานที่:
เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)
เซเว่นอีเลฟเว่น
เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2 (รหัสสาขา: 09797)
เซเว่นอิเลฟเว่น ถนนดินสอ (รหัสสาขา: 00117)
เซเว่นอีเลฟเว่น
เซเว่นอีเลฟเว่น
เซเว่นอีเลฟเว่น ราชดำเนิน (รหัสสาขา: 00216)
เซเว่นอีเลฟเว่น
เซเว่นอีเลฟเว่น ราชดำเนิน 3 (รหัสสาขา: 01102)
เซเว่นอิเลฟเว่น ตรอกมหรรณพาราม (รหัสสาขา: 15076)
เซเว่นอิเลฟเว่น สะพานวันชาติ (รหัสสาขา: 04217)
เซเว่นอีเลฟเว่น
เซเว่นอิเลฟเว่น ถนนราชดำเนินกลาง (รหัสสาขา: 10866)
เซเว่นอิเลฟเว่น Booth ตะนาว (รหัสสาขา: 00585)
เซเว่นอีเลฟเว่น
เซเว่นอิเลฟเว่น ถนนมหรรณพ (รหัสสาขา: 04249)
เซเว่นอิเลฟเว่น Booth ข้าวสาร (รหัสสาขา: 01355)
เซเว่นอีเลฟเว่น ตรอกข้าวสาร (รหัสสาขา: 01777)
เซเว่นอีเลฟเว่น
เซเว่นอิเลฟเว่น บางลำภู 4 (รหัสสาขา: 17222)


In [ ]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets
import requests

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            for i, guide in enumerate(guides):
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("\nข้อมูลเส้นทาง:")
    print(route_info)

    print("\nกำลังสร้างแผนที่...")
    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                # เรียกใช้ฟังก์ชัน get_route โดยใช้พิกัดที่เลือก
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# ฟังก์ชันที่ดึงข้อมูลเส้นทาง
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?"
    params = {
        "flon": user_lon,  # ลองจิจูดของผู้ใช้
        "flat": user_lat,  # ละติจูดของผู้ใช้
        "tlon": dest_lon,  # ลองจิจูดของปลายทาง
        "tlat": dest_lat,  # ละติจูดของปลายทาง
        "mode": "t",  # โมเดลการเดินทาง
        "restrict": 0,
        "locale": "th",
        'key': os.getenv("key_longdo")  # ใช้ API Key จริง
    }

    try:
        response = requests.get(url, params=params)
        
        # ตรวจสอบว่า API ส่งสถานะ 200 (OK) หรือไม่
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                return data
            else:
                print(f"ข้อมูลเส้นทางไม่พบในผลลัพธ์: {data}")
                return None
        else:
            print(f"เกิดข้อผิดพลาดจาก API: {response.status_code}, Message: {response.text}")
            return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ API: {e}")
        return None

# ฟังก์ชันประมวลผลคำถามผู้ใช้และค้นหาสถานที่
# (สมมติว่ามีฟังก์ชัน `process_user_query` อยู่แล้ว)
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("\nรายชื่อสถานที่:")
        display_places_list(places_from_api, user_location)
    else:
        print("\nไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("\nไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น

รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)', (13.75523416, 1…


คุณเลือกสถานที่: เซเว่นอิเลฟเว่น Booth ตะนาว (รหัสสาขา: 00585)
ละติจูด: 13.754489, ลองจิจูด: 100.498435

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.498435, 'lat': 13.754489}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 10}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 135, 'interval': 28}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 55, 'interval': 10}, {'turn': 1, 'name': 'ซอยดำเนินกลางใต้', 'distance': 129, 'interval': 23}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 104, 'interval': 19}, {'turn': 0, 'name': 'ถนนตะนาว', 'distance': 151, 'interval': 59}, {'turn': 1, 'name': 'ถนนบุญศิริ', 'distance': 15, 'interval': 6}], 'fdistance': 0.9496057, 'tdistance': 7.2220855, 'distance': 628, 'interval': 155, 'penalty': 105, 'id': 31556166}]}

กำลังสร้างแผนที่...



คุณเลือกสถานที่: เซเว่นอิเลฟเว่น ตรอกมหรรณพาราม (รหัสสาขา: 15076)
ละติจูด: 13.7536271, ลองจิจูด: 100.50014747

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.50014747, 'lat': 13.7536271}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 10}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 135, 'interval': 28}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 55, 'interval': 10}, {'turn': 0, 'name': 'ซอยดำเนินกลางใต้', 'distance': 60, 'interval': 11}, {'turn': 1, 'name': 'ถนนเชื่อมต่อ', 'distance': 144, 'interval': 25}, {'turn': 0, 'name': 'ตรอกศิลป์', 'distance': 71, 'interval': 13}, {'turn': 1, 'name': 'ถนนดินสอ', 'distance': 179, 'interval': 54}, {'turn': 1, 'name': 'ถนนมหรรณพ', 'distance': 107, 'interval': 47}], 'fdistance': 0.9496057, 'tdistance': 9.430536, 'distance': 790, 'interval': 198, 'penalty': 120, 'id': 31556174}]}

ก


คุณเลือกสถานที่: เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2 (รหัสสาขา: 09797)
ละติจูด: 13.757852, ลองจิจูด: 100.502241

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.502241, 'lat': 13.757852}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 111, 'interval': 30}, {'turn': 2, 'name': 'ถนนดินสอ', 'distance': 106, 'interval': 9}], 'fdistance': 0.9496057, 'tdistance': 10.145972, 'distance': 217, 'interval': 39, 'penalty': 30, 'id': 39807313}]}

กำลังสร้างแผนที่...



คุณเลือกสถานที่: เซเว่นอิเลฟเว่น บางลำภู 4 (รหัสสาขา: 17222)
ละติจูด: 13.75976719, ลองจิจูด: 100.49904531

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.49904531, 'lat': 13.75976719}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 11}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 292, 'interval': 61}, {'turn': 1, 'name': 'ถนนตะนาว', 'distance': 306, 'interval': 74}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 8, 'interval': 1}], 'fdistance': 0.9496057, 'tdistance': 8.600729, 'distance': 645, 'interval': 147, 'penalty': 60, 'id': 39807320}]}

กำลังสร้างแผนที่...


In [ ]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets
import requests

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            for i, guide in enumerate(guides):
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("\nข้อมูลเส้นทาง:")
    print(route_info)

    print("\nกำลังสร้างแผนที่...")
    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                # เรียกใช้ฟังก์ชัน get_route โดยใช้พิกัดที่เลือก
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# ฟังก์ชันที่ดึงข้อมูลเส้นทาง
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?"
    params = {
        "flon": user_lon,  # ลองจิจูดของผู้ใช้
        "flat": user_lat,  # ละติจูดของผู้ใช้
        "tlon": dest_lon,  # ลองจิจูดของปลายทาง
        "tlat": dest_lat,  # ละติจูดของปลายทาง
        "mode": "t",  # โมเดลการเดินทาง
        "restrict": 0,
        "locale": "th",
        'key': os.getenv("key_longdo")  # ใช้ API Key จริง
    }

    try:
        response = requests.get(url, params=params)
        
        # ตรวจสอบว่า API ส่งสถานะ 200 (OK) หรือไม่
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                return data
            else:
                print(f"ข้อมูลเส้นทางไม่พบในผลลัพธ์: {data}")
                return None
        else:
            print(f"เกิดข้อผิดพลาดจาก API: {response.status_code}, Message: {response.text}")
            return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ API: {e}")
        return None



# ฟังก์ชันประมวลผลคำถามผู้ใช้และค้นหาสถานที่
# (สมมติว่ามีฟังก์ชัน `process_user_query` อยู่แล้ว)
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("\nรายชื่อสถานที่:")
        display_places_list(places_from_api, user_location)
    else:
        print("\nไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("\nไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น

รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)', (13.75523416, 1…


คุณเลือกสถานที่: เซเว่นอีเลฟเว่น ตรอกข้าวสาร (รหัสสาขา: 01777)
ละติจูด: 13.7585479999, ลองจิจูด: 100.498331

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.498331, 'lat': 13.7585479999}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 111, 'interval': 36}, {'turn': 2, 'name': 'ถนนดินสอ', 'distance': 280, 'interval': 98}, {'turn': 0, 'name': 'ถนนพระสุเมรุ', 'distance': 545, 'interval': 63}, {'turn': 0, 'name': 'ถนนจักรพงษ์', 'distance': 397, 'interval': 99}, {'turn': 0, 'name': 'ถนนข้าวสาร', 'distance': 318, 'interval': 177}], 'fdistance': 0.9496057, 'tdistance': 13.058486, 'distance': 1651, 'interval': 473, 'penalty': 75, 'id': 39860571}]}

กำลังสร้างแผนที่...



คุณเลือกสถานที่: เซเว่นอิเลฟเว่น ตรอกมหรรณพาราม (รหัสสาขา: 15076)
ละติจูด: 13.7536271, ลองจิจูด: 100.50014747

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.50014747, 'lat': 13.7536271}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 14}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 135, 'interval': 46}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 55, 'interval': 10}, {'turn': 0, 'name': 'ซอยดำเนินกลางใต้', 'distance': 60, 'interval': 11}, {'turn': 1, 'name': 'ถนนเชื่อมต่อ', 'distance': 144, 'interval': 25}, {'turn': 0, 'name': 'ตรอกศิลป์', 'distance': 71, 'interval': 13}, {'turn': 1, 'name': 'ถนนดินสอ', 'distance': 179, 'interval': 47}, {'turn': 1, 'name': 'ถนนมหรรณพ', 'distance': 107, 'interval': 34}], 'fdistance': 0.9496057, 'tdistance': 9.430536, 'distance': 790, 'interval': 200, 'penalty': 120, 'id': 31509157}]}

ก